In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_pickle('data.pkl')

In [ ]:
type(df)

dict

In [ ]:
list_ans =[]
for i, item in enumerate(df['answers']['answer_start']):
  dic = {'answer_start':[df['answers']['answer_start'][i]],'text':[df['answers']['text'][i]]}
  list_ans.append(dic)

df["answers"] = list_ans

In [ ]:
from datasets import Dataset, DatasetDict

#ds = DatasetDict()

ds = Dataset.from_dict(df)


In [ ]:
ds

Dataset({
    features: ['context', 'answers', 'question'],
    num_rows: 649
})

In [ ]:
ds['answers'][:10]

[{'answer_start': [33], 'text': ['the']},
 {'answer_start': [73], 'text': ['an']},
 {'answer_start': [86], 'text': ['is']},
 {'answer_start': [32], 'text': ['even']},
 {'answer_start': [23], 'text': ['hunger']},
 {'answer_start': [0], 'text': ['not']},
 {'answer_start': [88], 'text': ['should']},
 {'answer_start': [18], 'text': ['central']},
 {'answer_start': [31], 'text': ['move']},
 {'answer_start': [36], 'text': ['the']}]

In [ ]:
ds.features

{'context': Value(dtype='string', id=None),
 'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)},
 'question': Value(dtype='string', id=None)}

In [ ]:
ds_train_test = ds.train_test_split(test_size=0.1)
ds_final = ds_train_test["train"].train_test_split(test_size=0.1)
ds_final["val"] = ds_train_test["test"]

In [ ]:
ds_final

DatasetDict({
    train: Dataset({
        features: ['context', 'answers', 'question'],
        num_rows: 525
    })
    test: Dataset({
        features: ['context', 'answers', 'question'],
        num_rows: 59
    })
    val: Dataset({
        features: ['context', 'answers', 'question'],
        num_rows: 65
    })
})

In [ ]:
ds_final['train'].features

{'context': Value(dtype='string', id=None),
 'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)},
 'question': Value(dtype='string', id=None)}

In [ ]:
#from datasets import Sequence, Value
#ds_final["train"] = ds_final["train"].cast_column('answers', Sequence(feature={'answer_start':Value(dtype='int32'),'text':Value(dtype='string')}))
#ds_final["train"] = ds_final["train"].cast_column('answers', Sequence())

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=128,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1


        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_ds = ds_final.map(preprocess_function, batched=True, remove_columns=ds_final["train"].column_names)

Map:   0%|          | 0/525 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions'],
        num_rows: 525
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions'],
        num_rows: 59
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions'],
        num_rows: 65
    })
})

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_ds["train"], batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(
    tokenized_ds["val"], batch_size=8, collate_fn=data_collator)

In [ ]:
for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 128]),
 'attention_mask': torch.Size([8, 128]),
 'offset_mapping': torch.Size([8, 128, 2]),
 'start_positions': torch.Size([8]),
 'end_positions': torch.Size([8])}

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to

In [ ]:
for name, param in model.named_parameters():
  if(name.startswith("qa_outputs")):
    param.requires_grad = True
  else:
    param.requires_grad = False
for name, param in model.named_parameters():
  if(name.startswith("qa_outputs")):
    print(name)
model.num_parameters(only_trainable =True)

qa_outputs.weight
qa_outputs.bias


1538

In [ ]:
outputs = model(**batch)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([8, 128]) torch.Size([8, 128])


In [ ]:
import torch


def mask_data(batch,outputs):

  for iter, item in enumerate(batch['input_ids']):

    start_index = (torch.where(item == 102)[0][0]) + 1
    end_index = (torch.where(item == 102)[0][1])+1
    # Mask everything apart from the tokens of the context
    mask = torch.BoolTensor(item.shape)
    for index in range(0,len(item)):
      if index >= start_index and index < end_index:
        mask[index] = False
      else:
        mask[index] = True

    # Unmask the [CLS] token
    mask[0] = False
    outputs['start_logits'][iter][mask] = -10000
    outputs['end_logits'][iter][mask] = -10000


  return outputs



In [ ]:
outputs = mask_data(batch,outputs)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [ ]:
start_probabilities[0].argmax()

tensor(0)

In [ ]:
print(start_logits.shape)

print(start_probabilities.shape)

torch.Size([8, 128])
torch.Size([8, 128])


In [ ]:
def compute_acc(start_probabilities,batch):
  acc = 0
  for index, item in enumerate(start_probabilities):
    max_prob_index = start_probabilities[index].argmax()
    if max_prob_index == batch['start_positions'][index]:
      acc += 1

  return acc/start_probabilities.shape[0]


In [ ]:
device  =  "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model  = model.to(device)

In [ ]:
# creating training step

from typing import Tuple

def train_step(
    model : torch.nn.Module,
    data_loader : torch.utils.data.DataLoader,
    loss_fn : torch.nn.Module,
    optimizer : torch.optim.Optimizer,
    device : torch.device ) -> Tuple[float,float]:



    # setup train loss and train accuracy values
    train_loss, train_acc = 0,0

    # put the model in training mode
    model.train()

    # loop through dataloader data batches
    for batch in data_loader:
      #print({k: v.shape for k, v in batch.items()})
      #print(type(batch["input_ids"]))
      # send data to target device
      #X, y = X['pixel_values'][0].to(device), y.to(device)

      #forward pass
      batch= {'input_ids':batch['input_ids'].to(device),'attention_mask':batch['attention_mask'].to(device),
              'start_positions':batch['start_positions'].to(device),'end_positions':batch['end_positions'].to(device)}
      outputs = model(**batch)

      #compute loss and accumulate loss
      loss = outputs.loss
      train_loss += loss.item()

      #optimzer zerograd
      optimizer.zero_grad()

      #loss backward
      loss.backward()

      #update parameters
      optimizer.step()

      # compute accuracy and accumulate accuracy
      outputs = mask_data(batch,outputs)
      start_logits = outputs.start_logits
      start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
      acc_batch = compute_acc(start_probabilities,batch)
      train_acc += acc_batch


    #adjust matrics to get average loss and accuracy per epochs
    train_loss = train_loss / len(data_loader)
    train_acc = train_acc / len(data_loader)

    return train_loss, train_acc

In [ ]:
def test_step(
    model : torch.nn.Module,
    data_loader : torch.utils.data.DataLoader,
    loss_fn : torch.nn.Module,
    device :  torch.device) -> Tuple[float, float]:

    #put model into evaluation mode
    model.eval()

    # initialize test loss and acc
    test_loss, test_acc = 0,0

    #turn on inference context manager
    with torch.inference_mode():
      # loop through batches
      for batch in data_loader:

        # move to device
        #X,y = X['pixel_values'][0].to(device),y.to(device)

        #compute logits
        batch= {'input_ids':batch['input_ids'].to(device),'attention_mask':batch['attention_mask'].to(device),
              'start_positions':batch['start_positions'].to(device),'end_positions':batch['end_positions'].to(device)}
        outputs = model(**batch)


        #compute loss and accumulate
        loss = outputs.loss
        test_loss += loss.item()

        #compute accuracy and accumulate
        outputs = mask_data(batch,outputs)
        start_logits = outputs.start_logits
        start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
        acc_batch = compute_acc(start_probabilities,batch)
        test_acc += acc_batch

    test_loss = test_loss / len(data_loader)
    test_acc = test_acc /len(data_loader)

    return test_loss,test_acc

In [ ]:
from typing import Dict, List

from tqdm.auto import tqdm

def train(
    model : torch.nn.Module,
    train_dataloader : torch.utils.data.DataLoader,
    test_dataloader : torch.utils.data.DataLoader,
    optimizer : torch.optim.Optimizer,
    loss_fn :torch.nn.Module,
    epochs : int,
    device : torch.device ) -> Dict[str,List[float]]:

    # create empty results dictionary
    results = {
               "train_loss" :[],
               "train_acc" :[],
               "test_loss" :[],
               "test_acc"  :[]  }

    # loop through training and test data for a number of  epochs
    for epoch in tqdm(range(epochs)):

      train_loss, train_acc = train_step(model=model,data_loader = train_dataloader, optimizer= optimizer, loss_fn =loss_fn,device=device)

      test_loss, test_acc = test_step(model = model,data_loader = test_dataloader, loss_fn = loss_fn, device=device)

      # printout loss and acc
      print(
          f"Epoch : {epoch+1} |"
          f"Train loss : {train_loss:.4f} |"
          f"Train acc : {train_acc:.4f} |"
          f"Test loss : {test_loss:.4f} |"
          f"Test acc : {test_acc:.4f}"
      )

      # update the results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)


    return  results

In [ ]:
# set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# set number of epochs
NUM_EPOCHS = 50


# Setup loss function and optimizer

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = .001)

# start the timer

from timeit import default_timer as timer
start_time = timer()

# train model_0

model_0_results = train(
    model = model,
    train_dataloader=train_dataloader,
    test_dataloader= eval_dataloader,
    epochs = NUM_EPOCHS,
    optimizer = optimizer,
    loss_fn = loss_fn,
    device = device
)


# end the timer and print the total time
end_time = timer()
print(f"[INFO] total training time : {end_time-start_time:.3f} seconds")

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch : 1 |Train loss : 1.4795 |Train acc : 0.5841 |Test loss : 1.7900 |Test acc : 0.5417
Epoch : 2 |Train loss : 1.4851 |Train acc : 0.5784 |Test loss : 1.7904 |Test acc : 0.5556
Epoch : 3 |Train loss : 1.5935 |Train acc : 0.5708 |Test loss : 1.8078 |Test acc : 0.5556
Epoch : 4 |Train loss : 1.4248 |Train acc : 0.5962 |Test loss : 1.7989 |Test acc : 0.5278
Epoch : 5 |Train loss : 1.5050 |Train acc : 0.5765 |Test loss : 1.8032 |Test acc : 0.5417
Epoch : 6 |Train loss : 1.5043 |Train acc : 0.5583 |Test loss : 1.7994 |Test acc : 0.5417
Epoch : 7 |Train loss : 1.4733 |Train acc : 0.5905 |Test loss : 1.8096 |Test acc : 0.5417
Epoch : 8 |Train loss : 1.4939 |Train acc : 0.5886 |Test loss : 1.8027 |Test acc : 0.5556
Epoch : 9 |Train loss : 1.4827 |Train acc : 0.5727 |Test loss : 1.7921 |Test acc : 0.5417
Epoch : 10 |Train loss : 1.5189 |Train acc : 0.5697 |Test loss : 1.8073 |Test acc : 0.5556
Epoch : 11 |Train loss : 1.4403 |Train acc : 0.5856 |Test loss : 1.7943 |Test acc : 0.5556
Epoch : 

In [ ]:
test_dataloader = DataLoader(
    tokenized_ds["test"], shuffle=True, batch_size=5, collate_fn=data_collator
)

for batch in test_dataloader:
  break


In [ ]:
for batch in test_dataloader:
  break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([5, 128]),
 'attention_mask': torch.Size([5, 128]),
 'offset_mapping': torch.Size([5, 128, 2]),
 'start_positions': torch.Size([5]),
 'end_positions': torch.Size([5])}

In [ ]:
def find_ans(outputs,batch,tokenizer):
  start_logits = outputs.start_logits
  #end_logits = outputs.end_logits
  start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
  #end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)
  words = []
  for index, item in enumerate(start_probabilities):
    #indices = []
    #max_prob_index_start = start_probabilities[index].argmax()
    indices_top5 = sorted(range(len(start_probabilities[index])), key=lambda i: start_probabilities[index][i])[-5:]
    tokens = [batch['input_ids'][index][i:i+1] for i in indices_top5 ]
    #indices.append( max_prob_index_start)
    #indices.append( max_prob_index_end)
      # tokens = batch['input_ids'][index][max_prob_index_start:max_prob_index_start+1]
    word = [tokenizer.decode(i) for i in  tokens]
    #else:
       #word = "###"
    words.append(word)
    print()

  return words


  #sorted(range(len(a)), key=lambda i: a[i])[-2:]
  #sorted(range(len(start_probabilities[index])), key=lambda i: start_probabilities[index][i])[-5:]

In [ ]:
def print_data(sentences,words):
   for index, item in enumerate(sentences):
    print(item)
    print(words[index])


In [ ]:
batch_new= {'input_ids':batch['input_ids'].to(device),'attention_mask':batch['attention_mask'].to(device),
            'start_positions':batch['start_positions'].to(device),'end_positions':batch['end_positions'].to(device)}
outputs = model(**batch_new)
outputs = mask_data(batch_new,outputs)
#start_logits = outputs.start_logits
#end_logits = outputs.end_logits
#start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
#end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)
words = find_ans(outputs,batch_new,tokenizer)
sentences = tokenizer.batch_decode(batch_new['input_ids'],skip_special_tokens=True)
print_data(sentences,words)






back? mutual value between the rupee and the rouble was mooted but this would hark india back to the pre liberalisation
['back', 'the', 'to', 'and', 'was']
there? construction is not allowed will be provided land in the first phase however the administration remained silent on whether there
['construction', 'is', 'be', 'there', 'provided']
more? will cost the state roughly the same as the anna bhagya promise more importantly compared with the cash transfer to
['the', 'more', 'cost', 'the', 'the']
of? energy sector and won the case however over time some of the sub federal schemes expired or were modified substantially
['substantially', 'of', 'the', 'over', 'were']
u? u s will receive and consider requests for exclusions from section measures with approval rates of for steel products and
['for', 'from', 'and', 'receive', 'u']


In [ ]:
# real world data
context = "The character of our republic is being severely assaulted in a systematic manner by the BJP. We are at a most crucial juncture in our nation's history."
question = "assaulted?"

In [ ]:
tokenized_input = tokenizer(question,context)
tokenized_input['input_ids'] = torch.tensor([tokenized_input['input_ids']]).to(device)
tokenized_input['attention_mask'] = torch.tensor([tokenized_input['attention_mask']]).to(device)

In [ ]:
output= model(**tokenized_input)
output = mask_data(tokenized_input,output)
words = find_ans(output,tokenized_input,tokenizer)
sentences = tokenizer.batch_decode(tokenized_input['input_ids'],skip_special_tokens=True)
print_data(sentences,words)


assaulted? the character of our republic is being severely assaulted in a systematic manner by the bjp. we are at a most crucial juncture in our nation's history.
['our', 'is', 'are', 'being', 'assaulted']


In [ ]:
!pip install requests

In [ ]:
import requests
api = "https://api.dictionaryapi.dev/api/v2/entries/en/"
input = api + "ask"
response = requests.get(f"{input}")
response.json()[0]["meanings"][0]["definitions"][0]['definition']

'An act or instance of asking.'

In [ ]:
# creating function to save the model

from pathlib import Path

def save_model(
    model : torch.nn.Module,
    target_dir :str,
    model_name : str):

  # create target directory

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok =True)

  # create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt") , " model name shoulf ends with .pt or .pth"
  model_save_path = target_dir_path / model_name

  # save the model state_dict
  print(f"[INFO] saving the model to : {model_save_path}")
  #torch.save(obj = model.state_dict(),f = model_save_path)
  torch.save(obj = model,f = model_save_path)

In [ ]:
# save the model
save_model(model = model, target_dir= "models", model_name ="qa.pth")

[INFO] saving the model to : models/qa.pth


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 7.8 MB/s eta 0:00:00
  

In [ ]:
import gradio as gr

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
import requests
api = "https://api.dictionaryapi.dev/api/v2/entries/en/"
import torch
model = torch.load("models/qa.pth")
model.eval()
device = "cuda"

def mask_data(batch,outputs):

  for iter, item in enumerate(batch['input_ids']):

    start_index = (torch.where(item == 102)[0][0]) + 1
    end_index = (torch.where(item == 102)[0][1])+1
    # Mask everything apart from the tokens of the context
    mask = torch.BoolTensor(item.shape)
    for index in range(0,len(item)):
      if index >= start_index and index < end_index:
        mask[index] = False
      else:
        mask[index] = True

    # Unmask the [CLS] token
    mask[0] = False
    outputs['start_logits'][iter][mask] = -10000
    outputs['end_logits'][iter][mask] = -10000


  return outputs


def find_ans(outputs,batch,tokenizer):
  start_logits = outputs.start_logits
  start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
  words = []
  for index, item in enumerate(start_probabilities):
    indices_top5 = sorted(range(len(start_probabilities[index])), key=lambda i: start_probabilities[index][i])[-5:]
    tokens = [batch['input_ids'][index][i:i+1] for i in indices_top5 ]
    word = [tokenizer.decode(i) for i in  tokens]
    words.append(word)

  return words

def ans_predict(context,question):

    tokenized_input = tokenizer(question,context)
    tokenized_input['input_ids'] = torch.tensor([tokenized_input['input_ids']]).to(device)
    tokenized_input['attention_mask'] = torch.tensor([tokenized_input['attention_mask']]).to(device)
    output= model(**tokenized_input)
    output = mask_data(tokenized_input,output)
    words = find_ans(output,tokenized_input,tokenizer)
    return words

def qa(context,question):

    ques = question.split(' ')[-1]
    words = ans_predict(context,ques)
    ques = ques.strip("?")
    flag = 0
    for index, word in enumerate(words[0]):
      if ques == word:
        flag = 1
        break
    if flag :
      input = api + ques
      response = requests.get(f"{input}")
      meaning = response.json()[0]["meanings"][0]["definitions"][0]['definition']
      dic = {}
      dic["text"] = context
      dic["entities"] = []
      pos_start = context.find(ques)
      pos_end = pos_start + len(ques)
      item = {}
      item["entity"] = meaning
      item["start"] = pos_start
      item["end"] = pos_end
      dic["entities"].append(item)
      return dic

    else:
      return "Couldn't find the word"


iface = gr.Interface(fn=qa,
                     inputs= [gr.Textbox(
                                    label="Passage",
                                    lines=3,
                                    ),
                              gr.Textbox(
                                    label="Question",
                                    lines=1,
                                    ),

                              ],
                      outputs = gr.HighlightedText(),
                      examples=[
                                                ["The character of our republic is being severely assaulted."],
                                                ["There is a deliberate attempt to weaken the federal structure of our polity"],
                                                ["We must build a fair economy with a strong and strategic public sector"],
                                                ["What is the meaning of federal?"],
                                                ["Meaning of economy?"]
                                             ]
                     )
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
import requests
api = "https://api.dictionaryapi.dev/api/v2/entries/en/"
import torch
model = torch.load("models/qa.pth")
model.eval()
device = "cuda"

def mask_data(batch,outputs):

  for iter, item in enumerate(batch['input_ids']):

    start_index = (torch.where(item == 102)[0][0]) + 1
    end_index = (torch.where(item == 102)[0][1])+1
    # Mask everything apart from the tokens of the context
    mask = torch.BoolTensor(item.shape)
    for index in range(0,len(item)):
      if index >= start_index and index < end_index:
        mask[index] = False
      else:
        mask[index] = True

    # Unmask the [CLS] token
    mask[0] = False
    outputs['start_logits'][iter][mask] = -10000
    outputs['end_logits'][iter][mask] = -10000


  return outputs


def find_ans(outputs,batch,tokenizer):
  start_logits = outputs.start_logits
  start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
  words = []
  for index, item in enumerate(start_probabilities):
    indices_top5 = sorted(range(len(start_probabilities[index])), key=lambda i: start_probabilities[index][i])[-5:]
    tokens = [batch['input_ids'][index][i:i+1] for i in indices_top5 ]
    word = [tokenizer.decode(i) for i in  tokens]
    words.append(word)

  return words

def ans_predict(context,question):

    tokenized_input = tokenizer(question,context)
    tokenized_input['input_ids'] = torch.tensor([tokenized_input['input_ids']]).to(device)
    tokenized_input['attention_mask'] = torch.tensor([tokenized_input['attention_mask']]).to(device)
    output= model(**tokenized_input)
    output = mask_data(tokenized_input,output)
    words = find_ans(output,tokenized_input,tokenizer)
    return words

def qa(context,question):

    ques = question.split(' ')[-1]
    words = ans_predict(context,ques)
    ques = ques.strip("?")
    flag = 0
    for index, word in enumerate(words[0]):
      if ques == word:
        flag = 1
        break
    if flag :
      input = api + ques
      response = requests.get(f"{input}")
      meaning = response.json()[0]["meanings"][0]["definitions"][0]['definition']
      dic = {}
      dic["text"] = context
      dic["entities"] = []
      pos_start = context.find(ques)
      pos_end = pos_start + len(ques)
      item = {}
      item["entity"] = meaning
      item["start"] = pos_start
      item["end"] = pos_end
      dic["entities"].append(item)
      return dic

    else:
      return "Couldn't find the word"




In [ ]:
# real world data
context = "The character of our republic is being severely assaulted in a systematic manner by the BJP. We are at a most crucial juncture in our nation's history."
question = "what is the meaning of assaulted?"

In [ ]:
qa(context,question)

[['our', 'is', 'are', 'being', 'assaulted']]


{'text': "The character of our republic is being severely assaulted in a systematic manner by the BJP. We are at a most crucial juncture in our nation's history.",
 'entities': [{'entity': 'To attack, physically or figuratively.',
   'start': 48,
   'end': 57}]}